# Welcome to point source imaging with COSIpy-classic
In this notebook, we'll use a Richardson-Lucy deconvolution algorithm to image four point sources: the Crab, Cyg X-1, Cen A, and Vela. This analysis requires significant computer memory (>50 GB), so you may want to use a more resource-intensive computer for this work. Please refer to the README for additional information on each step of the analysis.

## Import packages
We will need to import the cosipy-classic functions from COSIpy_dc1.py, response_dc1, and COSIpy_tools_dc1, as well as some other standard Python packages 

In [ ]:
from COSIpy_dc1 import *
import response_dc1
from COSIpy_tools_dc1 import *
from tqdm.autonotebook import tqdm
from numba import set_num_threads

import pickle

# set parallelism for whole notebook
set_num_threads(8)

### For the modified RL algorithm implemented here, we need to define a jaxopt objective function that fits background plus two images (the current image plus a delta image given by the RL formalism)


In [ ]:
import jax.config
import jax.numpy as jnp
import jax.scipy.stats as jstats
import jaxopt

# to better match Stan's behavior
jax.config.update("jax_enable_x64", True)

# objective function for MLE
def objective(params, data):
        (Abg, flux) = params
        (expc, cdelta, bg_model, bg_idx_arr, y, mu_flux, sigma_flux, mu_Abg, sigma_Abg) = data
 
        M = Abg[bg_idx_arr[:,None]] * bg_model + flux[0] * expc + flux[1] * cdelta

        # ensure that we don't accidentally use negative Possion means, which blows up likelihood
        M = jnp.maximum(M, 0)
        
        lp = jnp.sum(jstats.poisson.logpmf(y, M), axis=None) + \
             jnp.sum(jstats.norm.logpdf(flux, mu_flux, sigma_flux)) + \
             jnp.sum(jstats.norm.logpdf(Abg, mu_Abg, sigma_Abg))
        
        return -lp  # minimize to maximize LL

#opt = { 'disp': True }
optimizer = jaxopt.ScipyBoundedMinimize(fun=objective, method="l-bfgs-b", tol=1e-10)#, options=opt)

## Define file names
The 'Point_sources_10x_BG.tra.gz' file contains simulations of all four point sources, each at 10X their true flux, and Ling background. This is our data file.

You can optionally image only the point sources (without background) by changing this file to point source-only simulation. You will have to adjust the RL algorithm parameters later in the notebook.

In [ ]:
data_dir = '../data_products' # directory containing data & response files
filename = 'Point_sources_10x_BG.tra.gz' # combined simulation
response_filename = data_dir + '/Continuum_imaging_response.npz' # detector response
background_filename = data_dir + '/Scaled_Ling_BG_1x.npz' # background response
background_mode = 'from file'

pklfname = "ptsrc.pkl"

## Read simulation and define analysis object
Read in the data set and create the main cosipy-classic “analysis1" object, which provides various functionalities to study the specified file. This cell usually takes a few minutes to run.

In [ ]:
try:
    analysis1 = pickle.load(open(pklfname,'rb'))
    
except:
    print("loading analysis dataset")
    analysis1 = COSIpy(data_dir, filename)
    analysis1.read_COSI_DataSet()
    with open(pklfname, 'wb') as f:
        pickle.dump(analysis1, f)

# Bin the data
Calling "get_binned_data()" may take several minutes, depending on the size of the dataset and the number of bins. Keep an eye on memory here: if your time bins are very small, for example, this could be an expensive operation.

In [ ]:
# Define the bin sizes
Delta_T = 1800 # time bin size in seconds
energy_bin_edges = np.array([150, 220, 325, 480, 520, 765, 1120, 1650, 2350, 3450, 5000]) # as defined in the response
pixel_size = 6. # as defined in the response

analysis1.dataset.time_binning_tags_fast(time_bin_size=Delta_T)
analysis1.dataset.init_binning(energy_bin_edges=energy_bin_edges, pixel_size=pixel_size) # initiate the binning
analysis1.dataset.get_binned_data_fast() # bin data

## Examine the shape of the binned data.
The binned data are contained in "analysis1.dataset.binned_data." This is a 4-dimensional object representing the 5 dimensions of the Compton data space: (time, energy, $\phi$, FISBEL).

The number of bins in each dimension are shown by calling "shape."

In [ ]:
print("time, energy, phi, fisbel")
print(analysis1.dataset.binned_data.shape)

In [ ]:
# Can print the width of each time bin and the total time
print(analysis1.dataset.times.times_wid)
print(analysis1.dataset.times.total_time)

## Plot raw spectrum & light curve

In [ ]:
analysis1.dataset.plot_lightcurve()

analysis1.dataset.plot_raw_spectrum()
plt.xscale('log')

# Define the pointing object with the cosipy pointing class.

In [ ]:
# definition of pointings (balloon stability + Earth rotation)
pointing1 = Pointing(dataset=analysis1.dataset)

## Visualize the path of the Crab through the field-of-view.
This isn't necessary for the imaging algorithm, but is illustrative in the case of a point source. Take the Crab as an example.

In [ ]:
l_crab, b_crab = 184.55746, -5.78436 # Location of the Crab in Galactic coordinates [deg]

source = 'Crab Nebula'

plt.plot(np.rad2deg(pointing1.zpoins)[:,0]+360, np.rad2deg(pointing1.zpoins[:,1]), 'ko', label='COSI zenith pointing')
plt.plot(l_crab, b_crab, '*r', markersize=10 , label=f'{source}')
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]')
plt.legend();

In [ ]:
# COSI's field of view extends ~60 deg from zenith, hence why the Zenith is labeled as +60 deg above the horizon
analysis1.plot_elevation([l_crab], [b_crab], [f'{source}'])

# Define the BG model

In [ ]:
# Ling BG simulation to model atmospheric background
background1 = BG(dataset=analysis1.dataset,mode=background_mode,filename=background_filename)

# Read in the Response Matrix

In [ ]:
# continuum response
rsp = response_dc1.SkyResponse(filename=response_filename,pixel_size=pixel_size) # read in detector response

## Exploring the shape of the data space
The shape of the response spans (Galactic latitude $b$, Galactic longitude $\ell$, Compton scattering angle $\phi$,  FISBEL, energy). There is 1 energy bin for the 511 keV response ("analysis1.dataset.energies.n_energy_bins"). This is why there is no fifth dimension for the energy printed below. The shape of the data and background objects span (time, Compton scattering angle, FISBEL).

In [ ]:
rsp.rsp.response_grid_normed_efinal.shape

In [ ]:
np.shape(analysis1.dataset.binned_data)

In [ ]:
np.shape(background1.bg_model)

# Imaging Setup

## Define a grid on the sky to make images

In [ ]:
# We define our sky-grid on a regular (pixel_size x pixel_size) grid for testing (later finer grid)
binsize = np.deg2rad(pixel_size)

# Number of pixels in l and b
n_l = int(360/pixel_size)
n_b = int(180/pixel_size)

# Galactic coordiantes: l and b pixel edges
l_arrg = np.linspace(-np.pi,   np.pi,   n_l+1)
b_arrg = np.linspace(-np.pi/2, np.pi/2, n_b+1)

# Making a grid
L_ARRg, B_ARRg = np.meshgrid(l_arrg, b_arrg)

# Choosing the centre points as representative
l_arr = l_arrg[0:-1] + binsize/2
b_arr = b_arrg[0:-1] + binsize/2
L_ARR, B_ARR = np.meshgrid(l_arr, b_arr)

# Define solid angle for each pixel for normalisations later
domega    = binsize * np.diff(np.sin(b_arrg))  # per latitude
domegaMap = domega[:,None] # permit computing "map / domega" on n_b x n_l map

## Convert sky grid to zenith/azimuth pairs for all pointings

In [ ]:
# calculate the zeniths and azimuths on that grid for all times
coords = zenaziGrid_fast(pointing1.ypoins,
                         pointing1.xpoins,
                         pointing1.zpoins,
                         L_ARR.ravel(), B_ARR.ravel(),
                         pixel_size)

In [ ]:
# Reshape for next routines ... 
coords = coords.reshape(n_b, n_l, pointing1.dtpoins.size, 2)

## Get observation indices for non-zero bins
Here we also chose an energy bin to image. Energy bin "2" in the continuum response (and, necessarily, "energy_bin_edges" at the beginning of the notebook) is $320-480$ keV. We choose this energy bin because it has the highest count rate. Refer to the energy spectrum generated earlier.

In [ ]:
# Choose an energy bin to analyze
ebin = 2 # Analyzing 320-480 keV 
nonzero_idx = background1.calc_this[ebin]

## Reduce the response dimensions

In [ ]:
sky_response_CDS = rsp.rsp.response_grid_normed_efinal.reshape(
    n_b,
    n_l,
    analysis1.dataset.phis.n_phi_bins*\
    analysis1.dataset.fisbels.n_fisbel_bins, analysis1.dataset.energies.n_energy_bins)[:, :, nonzero_idx, ebin]

sky_response_CDS = np.ascontiguousarray(sky_response_CDS) # to speed up scaled response calculation
del rsp  # get rid of no-longer-needed full response, which is much larger than sky_response_CDS

In [ ]:
# reduced response dimensions:
# lat x lon x CDS
sky_response_CDS.shape

## Calculate the general response for the current data set
This has to be done only once (for the data set).

In [ ]:
from accelerate import get_image_response_from_pixelhit_general

sky_response_scaled = [] # clear out any old (large!) matrix if we are running this more than once

cut = 90 

sky_response_scaled = get_image_response_from_pixelhit_general(
    Response=sky_response_CDS,
    coords = coords,
    dt=pointing1.dtpoins,
    times_min=analysis1.dataset.times.times_min,
    n_ph_dx=analysis1.dataset.times.n_ph_dx,
    domega=domega,
    n_hours=analysis1.dataset.times.n_ph,
    pixel_size=pixel_size,
    cut=cut)
    #altitude_correction=False)

In [ ]:
# data-set-specific response dimensions
# times x lat x lon x CDS
sky_response_scaled.shape

## Exposure map
i.e. the response weighted by time

In [ ]:
from accelerate import emap_fast

expo_map = emap_fast(sky_response_scaled, n_b, n_l)

## Plotting the exposure map weighted with the pixel size

In [ ]:
plt.subplot(projection='aitoff')
p = plt.pcolormesh(L_ARRg,B_ARRg,np.roll(expo_map/domegaMap,axis=1,shift=0))
plt.contour(L_ARR,B_ARR,np.roll(expo_map/domegaMap,axis=1,shift=0),colors='black')
plt.colorbar(p, orientation='horizontal')

# Set up the RL algorithm

### Define function for a starting map for the RL deconvolution. We choose an isotropic map, i.e. all pixels on the sky are initialized with the same value

In [ ]:
def IsoMap(n_b, n_l, A0, domega):
    norm  = n_l * np.sum(domega)
    return A0/norm * np.ones((n_b, n_l))

### Number of time bins

In [ ]:
d2h = analysis1.dataset.binned_data.shape[0]
d2h

### Select only one energy bin (as above) for data set

In [ ]:
print('ebin: ',ebin)
dataset = analysis1.dataset.binned_data[:,ebin,:,:].reshape(d2h,
                                                            analysis1.dataset.phis.n_phi_bins*analysis1.dataset.fisbels.n_fisbel_bins)[:,nonzero_idx]
dataset = np.ascontiguousarray(dataset) # for performance in R-L

### Same for background

In [ ]:
background_model = background1.bg_model_reduced[ebin]
background_model = np.ascontiguousarray(background_model) # for performance in R-L

### Check for consistency of data and background
They must have the same dimensions. If not, the algorithm won't work.

In [ ]:
dataset.shape, background_model.shape

## Set an initial guess for the background amplitude
Feel free to play with this value, but here are suggestions informed by testing thus far:

### If source+BG:
We suggest setting "fitted_bg" to 0.9 or 0.99 when the loaded data/simulation (analysis1 object) contains both source and background. This is a rough estimate of the background contribution (90, 99%) to the entire data set.

### If analyzing source only:
When the analysis1 object does not contain background, we suggest setting this parameter to 1E-6, i.e. very close to zero background contribution.

In [ ]:
fitted_bg = np.array([0.9])

# Richardson-Lucy algorithm

## Individual steps are explained in the code.
The steps follow the algorithm as outlined in [Knoedlseder et al. 1999](https://ui.adsabs.harvard.edu/abs/1999A%26A...345..813K/abstract). Refer to that paper for a mathematical description of the algorithm.

In [ ]:
# Might not use this depending on if you choose to smooth the delta map

from scipy.ndimage import gaussian_filter

In [ ]:
from accelerate import convolve_fast, convdelta_fast
#import time

# Experiment with these variables!
#############################
# initial map (isotropic flat, small value)
map_init = IsoMap(n_b, n_l, 0.01, domega)

# number of RL iterations
maxiters = 150

# if MAP likelihood changes by less than this fraction in 1 iteration, terminate
ltol = 1e-9

# acceleration parameter
afl_scl = 2000.

# Define regions of the sky that we actually cannot see
# here we select everything, i.e. we have no bad exposure

bad_expo = np.where(expo_map/domegaMap <= 0)

#############################

######################################
# Initial sky map setup from map_init
######################################

# Current map starts as initial map
curr_map = map_init

# setting the map to zero where we selected a bad exposure (we didn't, but to keep it general)
curr_map[bad_expo] = 0

# check for each pixel to be finite (must be true for map_init)
assert not np.isnan(curr_map).any(), "NaNs in initial map!"

# convolve this map with the response
#print('Convolving with response (init expectation)')
#tstart = time.time()
curr_expectation = convolve_fast(sky_response_scaled, curr_map,
                                 sky_response_scaled.shape[0], sky_response_scaled.shape[3],
                                 n_b, n_l)
#tend = time.time()
#print(f'Time in convolution: {tend - tstart:.2f}s')

#########################################################
# Computations pulled out of loop
#########################################################

## Define background model cuts, indices, and resulting number of cuts
bg_cuts, idx_arr, Ncuts = background1.bg_cuts, background1.idx_arr, background1.Ncuts
 
# temporary background model
model_bg = background_model * fitted_bg[idx_arr, None]

# cf. Knoedlseder+1997 what the values denominator etc are
# this is the response R summed over the CDS and the time bins

# denominator scaled by fourth root to avoid exposure e#dge effects
# You can try changing 0.25 to 0, 0.5, for example
den_scale = 0.25

idenominator = expo_map**(-(1-den_scale))

#########################################################
# Storage for intermediate parameters
#########################################################

# maps per iteration
map_iterations = np.empty((maxiters, n_b, n_l))

# likelihood of maps (vs. initial, i.e., basically only background)
map_likelihoods = np.empty(maxiters)

# store per-iter fit likelihoods, i.e., fit quality
intermediate_lp = np.empty(maxiters)

# store per-iter acceleration parameters (lambda)
acc_par = np.empty(maxiters)

# store per-iter fitted background parameters 
bg_pars = np.empty((maxiters, Ncuts))

###########################################################
## iterative R-L loop                    
###########################################################

# expectation (in data space) is the image (curr_expectation) plus the background (model_bg)
curr_expectation_tot = curr_expectation + model_bg 

# save initial map
map_iterations[0,:,:] = curr_map

# save initial map's likelihood
map_likelihoods[0] = cashstat(dataset, curr_expectation_tot)

for its in tqdm(range(1, maxiters)):

    # calculate numerator of RL algorithm
   
    #print(f'Calculating Delta image, iteration {its}, numerator')
    #tstart = time.time()
    numerator = convdelta_fast(sky_response_scaled, dataset, curr_expectation_tot,
                               n_b, n_l, dataset.shape[0], dataset.shape[1])
    #tend = time.time()
    #print(f'Time in Delta image calc: {tend - tstart:.2f}s')
    
    delta_map_tot = curr_map * numerator * idenominator
   
    # You can also try to smooth the delta map
    #curr_delta_map_tot = gaussian_filter(curr_delta_map_tot, 0.5)
        
    # zero our bad exposure regions
    delta_map_tot[bad_expo] = 0

    # should never happen
    delta_map_tot[np.isnan(delta_map_tot)] = 0
            
    # convolve delta image
    #print(f'Convolving Delta image, iteration {its}')
    #tstart = time.time()
    conv_delta_map_tot = convolve_fast(sky_response_scaled, delta_map_tot,
                                    sky_response_scaled.shape[0], sky_response_scaled.shape[3],
                                    n_b, n_l)
    #tend = time.time()
    #print(f'Time in convolution: {tend - tstart:.2f}s')

    # Find maximum acceleration parameter to multiply delta image with
    # so that the total image is still positive everywhere.
    # If there are no negative entries in delta_map_tot_old, there is no upper bound on the
    # acceleration.  Original code used a value of ~10000 in this case.  If we use much larger
    # value, RL seems to oscillate rather than converging smoothly and gives a worse final
    # likelihood (observed on the Point_Sources notebook, which is the only one with this issue).
    assert np.min(curr_map) >= 0, "current map contains negative entries!"
    neg = delta_map_tot < 0
    if not neg.any():
        afl = 10000
    else:
        afl = int(np.floor(np.min(-afl_scl * curr_map[neg] / delta_map_tot[neg]))) 
        afl = min(afl, 10000)
    
    print('Maximum acceleration parameter found: ', afl/afl_scl)

    # fit:
    
    cdelta = conv_delta_map_tot/afl_scl

    mu_Abg = fitted_bg    # can play with this
    sigma_Abg = fitted_bg # can play with this
    mu_flux = np.array([1,afl/2])
    sigma_flux = np.array([1e-2,afl])

    init_params =  (jnp.ones(Ncuts) * fitted_bg, jnp.array([1, afl/2.]))
    
    acceleration_factor_limit = afl * 0.95
    lower_bounds = (jnp.ones(Ncuts) * 1e-8,    jnp.ones(2) * 1e-8)
    upper_bounds = (jnp.ones(Ncuts) * jnp.inf, jnp.ones(2) * acceleration_factor_limit)
    
    #print('Optimizing bg parameters')
    #tstart = time.time()
    res = optimizer.run(init_params, bounds=(lower_bounds, upper_bounds),
                        data=(curr_expectation, cdelta, model_bg, idx_arr, dataset,
                              mu_flux, sigma_flux, mu_Abg, sigma_Abg))
    #tend = time.time()
    #print(f'Time in optimizer: {tend - tstart:.2f}s')

    if not res.state.success:
        print("*** Optimizer failed! rerun with options = { 'disp': True } to see error messages")
       
        # proceed with a safe acceleration <= 1 (safe = new map does not go negative at any pixel)
        print("proceeding with a safe acceleration parameter")
        accScale = np.minimum(1., acceleration_factor_limit)
    else:
        # save values
        #print(f'Saving new map, and fitted parameters, iteration {its}')
        intermediate_lp[its-1] = -res.state.fun_val
        
        newAbg, newflux = res.params
        newAcc = newflux[1]
        bg_pars[its-1,:] = newAbg
        acc_par[its-1]   = newAcc

        accScale = float(newAcc)/afl_scl
    
    # plot each iteration's map and its delta map here to match previous impl's behavior
    # (not required, but nice to see how the algorithm is doing)
    plt.figure(figsize=(16,6))
    plt.subplot(121)
    plt.pcolormesh(L_ARRg,B_ARRg,np.roll(curr_map, axis=1, shift=0)) 
    plt.colorbar()

    plt.subplot(122)
    plt.pcolormesh(L_ARRg,B_ARRg,np.roll(delta_map_tot, axis=1, shift=0)) 
    plt.colorbar()
    plt.show()

    # make new map as old map plus scaled delta map
    curr_map += accScale * delta_map_tot
    
    # setting the map to zero where we selected a bad exposure (we didn't, but to keep it general)
    curr_map[bad_expo] = 0
    
    # check for each pixel to be finite
    curr_map[np.isnan(curr_map)] = 0

    # save map
    map_iterations[its,:,:] = curr_map 

    # make new expectation as old expectation plus scaled conv_delta map
    curr_expectation += accScale * conv_delta_map_tot

    # expectation (in data space) is the image (expectation) plus the background (model_bg)
    curr_expectation_tot = curr_expectation + model_bg 

    # calculate likelihood of current total expectation
    map_likelihoods[its] = cashstat(dataset, curr_expectation_tot)

    # how much did the MAP likelihood improve since the prior iteration?
    dml = np.abs((map_likelihoods[its] - map_likelihoods[its-1])/map_likelihoods[its-1])

    print(f"After iteration {its}: MAP likelihood = {map_likelihoods[its]:.2f}, rel. change = {dml:.2e}")

    if dml < ltol:
        print(f"MAP likelihood change was less than {ltol} -- terminating")
        break

## Plot the fitted background parameter and the map flux

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.plot(range(its), [i[0] for i in bg_pars[:its]], '.-')
plt.xlabel('Iteration')
plt.ylabel('BG params]')


plt.subplot(122)
map_fluxes = np.zeros(its+1)
for i in range(its+1):
    map_fluxes[i] = np.sum(map_iterations[i,:,:]*domegaMap)
    
plt.plot(map_fluxes[:its],'o-')
plt.xlabel('Iteration')
plt.ylabel('Flux')# [ph/keV]')

## Did the algorithm converge? Look at the likelihoods.
intermediate_lp: Fit likelihoods, i.e. fit quality

map_likelihoods: likelihood of maps (vs. initial i.e. basically only background)

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.plot(np.arange(its), intermediate_lp[:its], '.-')
plt.xlabel('Iteration')
plt.ylabel('likelihood (intermediate_lp)')

plt.subplot(122)
plt.plot(range(its+1), map_likelihoods[:its+1], '.-')
plt.xlabel('Iteration')
plt.ylabel('likelihood (map_likelihoods)')

print(f'final MAP likelihood = {map_likelihoods[its]}')

## Make the image!
You can loop over all iterations to make a GIF or just show one iteration (usually the final iteration).

In [ ]:
from IPython.display import Image
from IPython.display import Video

from matplotlib import animation

from matplotlib import colors

from scipy.ndimage import gaussian_filter as smooth

In [ ]:
# Choose an image to plot
idx = its  # final image

In [ ]:
deg2rad = np.pi/180

# Choose a color map like viridis (matplotlib default), nipy_spectral, twilight_shifted, etc. Not jet.
cmap = plt.get_cmap('viridis') 

# Bad exposures will be gray
cmap.set_bad('lightgray')


##################
# Select here which pixels should be gray
map_iterations_nan = np.copy(map_iterations)

# Select also non-zero exposures here to be gray (avoiding the edge effects)
# You can play with this. Most success in testing with 1e4, 1e3
bad_expo = np.where(expo_map/domegaMap <= 1e4) 

map_iterations_nan[:, bad_expo[0], bad_expo[1]] = np.nan
#################    


# Set up the plot
fig, ax = plt.subplots(figsize=(10.24,7.68), subplot_kw={'projection':'aitoff'}, nrows=1, ncols=1)

ax.set_xticks(np.array([-120,-60,0,60,120])*deg2rad)
ax.set_xticklabels([r'$-120^{\circ}$'+'\n',
                            r'$-60^{\circ}$'+'\n',
                            r'$0^{\circ}$'+'\n',
                            r'$60^{\circ}$'+'\n',
                            r'$120^{\circ}$'+'\n'])
ax.tick_params(axis='x', colors='orange')

ax.set_yticks(np.array([-60,-30,0,30,60])*deg2rad)
ax.tick_params(axis='y', colors='orange')

plt.xlabel('Gal. Lon. [deg]')
plt.ylabel('Gal. Lat. [deg]')


# "ims" is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame
ims = []

# If you want to make a GIF of all iterations:
#for i in range(its+1):

# If you only want to plot one image:
for i in [idx]:

    ttl = plt.text(0.5, 1.01, f'RL iteration {i}', horizontalalignment='center', 
                   verticalalignment='bottom', transform=ax.transAxes)
    
    # Either gray-out bad exposure (map_iterations_nan) or don't mask (map_iterations)
    # Masking out bad exposure 
    image = map_iterations_nan[i,:,:]
    #image = map_iterations[i,:,:]

    img = ax.pcolormesh(L_ARRg,B_ARRg,
                        
                        # Can shift the image along longitude. Here, no shift.
                        np.roll(image, axis=1, shift=0),
            
                        # Optionally smooth with gaussian filter
                        #smooth(np.roll(image, axis=1, shift=0), 0.75/pixel_size),
                        
                        cmap=plt.cm.viridis,
                        
                        # Optionally set the color scale. Default: linear
                        #norm=colors.PowerNorm(0.33)
                       )
    ax.grid()
    
    ims.append([img, ttl])

cbar = fig.colorbar(img, orientation='horizontal')
cbar.ax.set_xlabel(r'[Arbitrary Units]')
    

# Can save a sole image as a PDF 
#plt.savefig(data_dir + f'images/511keV_RL_image_iteration{idx}.pdf', bbox_inches='tight')

# # Can save all iterations as a GIF
# ani = animation.ArtistAnimation(fig, ims, interval=200, blit=True, repeat_delay=0)
# ani.save(f'/home/jacqueline/511keV_RL_image_{idx}iterations.gif')

# What do we see?
The Crab nebula is the only easily visible source in this combined simulation of 10x flux Crab, 10x flux Cygnus X-1, 10x flux Centaurus A, 10x Vela, and 1x flux Ling background (scaled to the observed 2016 flight background level). 

You can play with the color scaling to try to enhance the appearance of the other sources. Vela is likely too dim to be seen, however. 

You can also try running this notebook without including the Ling background. Change the loaded .tra.gz file at the beginning, adjust RL parameters as necessary, and see if the four point sources are more easily resolved without background contamination!

As another suggestion, what happens if you run this notebook using the 511 keV response? The 1809 keV response? A different energy bin of the continuum response? 

You can try combining these four point sources and Ling BG with the 10x 511 keV and 10x $^{26}$Al simulations for a full combined imaging test, using all three response simulations too.